# Prueba de funciones


📥Importaciones 

In [6]:
#
# Importación del módulo warnings para controlar advertencias
import warnings
from typing import Tuple
import gc

# Importación de pandas y se asigna el alias 'pd' para su uso
import pandas as pd
import time
# Importación de funciones desde data_utils
from data_utils import ej_review_sentimiento

# Configuración de la omisión de advertencias
warnings.filterwarnings("ignore")
import pandas as pd
import pyarrow as pa

# Carga de la extensión autoreload y configuración para recargar automáticamente módulos
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


📦 Extracción de los conjuntos de datos

In [7]:
df_steam_games =   pd.read_parquet('../0 Dataset/steam_games_LISTO.parquet')
#df_user_items =   pd.read_parquet('./0 Dataset/user_items_LISTO.parquet')
#df_user_reviews = pd.read_parquet('./0 Dataset/user_review_LISTO.parquet')

##### 1 🌐 `developer(desarrollador: str)`:

Devuelve la cantidad de juegos y porcentaje de contenido Free por año según empresa desarrolladora

 **📂 steam_games.json**: developer, release_date, price

🌐Para acelerar la velocidad de la api, he decidido crear un DF especifico. 

In [10]:
df_API_developer = df_steam_games[["developer","id", "release_year", "price"]]
df_API_developer.to_parquet("../0 Dataset/df_API_developer.parquet")

In [11]:
def developer(desarrollador: str):
    df_dev = df_steam_games[df_steam_games['developer'] == desarrollador]

    grouped = df_dev.groupby('release_year').agg(
        items=('id', 'count'),  # Cambiar de (x == 0).sum() a 'count' o 'sum'
        gratis=('price', lambda x: (x == 0).sum())
    )

    result = []
    for year, row in grouped.iterrows():
        result.append({
            "Año": int(year),
            "Juegos": int(row["items"]),
            "Gratis %": row["gratis"] / row["items"] * 100
        })
    
    # Llamamos al recolector de basura
    gc.collect()

    return result

In [12]:
# Example usage
resultado = developer('Valve')
print(resultado)

[{'Año': 1998, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 1999, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2000, 'Juegos': 2, 'Gratis %': 0.0}, {'Año': 2001, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2003, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2004, 'Juegos': 5, 'Gratis %': 0.0}, {'Año': 2005, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2006, 'Juegos': 2, 'Gratis %': 0.0}, {'Año': 2007, 'Juegos': 3, 'Gratis %': 0.0}, {'Año': 2008, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2009, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2010, 'Juegos': 2, 'Gratis %': 50.0}, {'Año': 2011, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2012, 'Juegos': 2, 'Gratis %': 0.0}, {'Año': 2014, 'Juegos': 7, 'Gratis %': 0.0}, {'Año': 2016, 'Juegos': 1, 'Gratis %': 100.0}, {'Año': 2017, 'Juegos': 2, 'Gratis %': 0.0}]


* Esta función desarrollador se encarga de analizar la información de los juegos en la base de datos df_steam_games, específicamente aquellos desarrollados por el desarrollador proporcionado como argumento.

* En resumen, la función realiza lo siguiente:

* Filtra los juegos por el desarrollador especificado.
* Cuenta la cantidad total de juegos por año.
* Cuenta la cantidad de juegos gratuitos por año.
* Calcula el porcentaje de juegos gratuitos sobre el total de juegos por cada año.
* Reemplaza los valores NaN (no disponibles) en el porcentaje de juegos gratuitos por 0%.
* Crea un DataFrame con los resultados, incluyendo el año, la cantidad total de juegos y el porcentaje de juegos gratuitos.
* Ordena el DataFrame por año.
* Formatea el porcentaje de juegos gratuitos para que se muestre con el símbolo de porcentaje (%).
* Devuelve el DataFrame resultante.

##### 🌐 userdata(User_id: str) : 
Dinero gastado por el usuario, porcentaje de recomendación y cantidad de items.

1. 📂 **user_reviews.json**: user_id, recommend, helpful.total
2. 📂 **users_items.json**: user_id, items_count

In [13]:
def userdata(user_id):

    # Filtra por el usuario de interés
    usuario = df_reviews[df_reviews['user_id'] == user_id]
    # Calcula la cantidad de dinero gastado para el usuario de interés
    cantidad_dinero = df_gastos_items[df_gastos_items['user_id']== user_id]['price'].iloc[0]
    # Busca el count_item para el usuario de interés    
    count_items = df_gastos_items[df_gastos_items['user_id']== user_id]['items_count'].iloc[0]
    
    # Calcula el total de recomendaciones realizadas por el usuario de interés
    total_recomendaciones = usuario['reviews_recommend'].sum()
    # Calcula el total de reviews realizada por todos los usuarios
    total_reviews = len(df_reviews['user_id'].unique())
    # Calcula el porcentaje de recomendaciones realizadas por el usuario de interés
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
    
    return {
        'cantidad_dinero': int(cantidad_dinero),
        'porcentaje_recomendacion': round(float(porcentaje_recomendaciones), 2),
        'total_items': int(count_items)
    }

'''


SyntaxError: incomplete input (2207087214.py, line 23)

In [ ]:

"""
_______________________________
1 DEF userdata(User_id: str):  

devolver cantidad de dinero gastado por el usuario
el porcentaje de recomendación en base a reviews.recommend 
y cantidad de items. 
_______________________________
"""
import pandas as pd
from typing import Dict

def userdata(User_id: str) -> Dict[str, float]:
    """
    Devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y la cantidad de items.
    
    Parameters:
    User_id (str): Identificador único del usuario.
    
    Returns:
    Dict[str, float]: Diccionario con los datos del usuario (dinero gastado, porcentaje de recomendación, cantidad de items).
    """
    # Leer los archivos Parquet
    user_reviews_df = pd.read_parquet('user_reviews.parquet')
    users_items_df = pd.read_parquet('users_items.parquet')
    
    # Obtener los datos del usuario
    user_data = users_items_df[users_items_df['user_id'] == User_id].iloc[0]
    
    # Obtener las revisiones del usuario
    user_reviews = user_reviews_df[user_reviews_df['user_id'] == User_id]
    
    # Calcular el total de dinero gastado y el porcentaje de recomendación
    total_spent = user_reviews['helpful.total'].sum()
    recommend_count = len(user_reviews[user_reviews['recommend']])
    total_reviews = len(user_reviews)
    recommend_percentage = (recommend_count / total_reviews) * 100 if total_reviews > 0 else 0.0
    
    return {
        "Dinero gastado": total_spent,
        "% de recomendación": recommend_percentage,
        "cantidad de items": user_data['items_count']
    }
